In [4]:
# Step 2. 데이터 전처리
# 한국어는 특수문자 처리가 중요합니다. 질문(Q)과 답변(A) 쌍을 추출하고 정제하는 과정을 거칩니다.
import pandas as pd
import re

# [루브릭 3번: 한국어 전처리를 통해 학습 데이터셋 구축]
def preprocess_sentence(sentence):
    # 양끝 공백 제거
    sentence = sentence.strip()
    # 구두점 앞에 공백을 추가하여 단어와 분리 (예: "반가워요!" -> "반가워요 !")
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 한글, 숫자, 영어, 기본 구두점 제외하고 모두 제거
    sentence = re.sub(r"[^가-힣?.!,0-9a-zA-Z]+", " ", sentence)
    return sentence.strip()

# 데이터 로드
data = pd.read_csv('~/work/transformer_chatbot/data/ChatbotData.csv')

# 질문과 답변 리스트 생성 및 전처리 적용
questions = [preprocess_sentence(q) for q in data['Q']]
answers = [preprocess_sentence(a) for a in data['A']]

print(f"전처리 후 질문 샘플: {questions[0]}")
print(f"전처리 후 답변 샘플: {answers[0]}")

전처리 후 질문 샘플: 12시 땡 !
전처리 후 답변 샘플: 하루가 또 가네요 .


In [6]:
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.2 MB/s eta 0:00:00


In [7]:
# Step 3. SentencePiece 사용하기
# 형태소 분석기 대신 Google의 SentencePiece를 사용하여 서브워드 토크나이징을 진행합니다. 이는 신조어나 복합어도 유연하게 처리할 수 있는 장점이 있습니다.
import sentencepiece as spm

# [루브릭 3번: 토크나이징 및 병렬 데이터 구축]
# SentencePiece 학습을 위해 텍스트 파일 생성
with open('chatbot.txt', 'w', encoding='utf-8') as f:
    for line in questions + answers:
        f.write(line + '\n')

# SentencePiece 모델 학습 (어휘 사전 크기 8000)
spm.SentencePieceTrainer.Train('--input=chatbot.txt --model_prefix=korean_spm --vocab_size=8000')
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# 시작 토큰(BOS)과 종료 토큰(EOS) 번호 정의 (보통 0, 1, 2는 예약어)
# SentencePiece 설정에 따라 다를 수 있으나 여기서는 직접 ID를 할당하여 정수 인코딩 진행
def encode(sentence):
    return [s.bos_id()] + s.EncodeAsIds(sentence) + [s.eos_id()]

# 모든 질문과 답변을 정수 시퀀스로 변환
tokenized_questions = [encode(q) for q in questions]
tokenized_answers = [encode(a) for a in answers]

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=chatbot.txt --model_prefix=korean_spm --vocab_size=8000
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: chatbot.txt
  input_format: 
  model_prefix: korean_spm
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piec

In [9]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 40.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 79.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22/22 [tensorflow]2 [tensorflow]]data-server]


In [16]:
# Step 3-2. 패딩 작업 추가하기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# [루브릭 3번: 데이터셋 구축을 위한 패딩 처리]

# 모든 시퀀스의 길이를 MAX_LENGTH(40)으로 통일합니다.
# 부족한 부분은 뒤(post)에 0을 채웁니다.
tokenized_questions = pad_sequences(tokenized_questions, maxlen=MAX_LENGTH, padding='post')
tokenized_answers = pad_sequences(tokenized_answers, maxlen=MAX_LENGTH, padding='post')

print("패딩 완료! 데이터의 크기(shape):", tokenized_questions.shape)

# 이제 다시 데이터셋을 생성합니다. (에러가 났던 그 코드입니다)
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': tokenized_questions,
        'dec_inputs': tokenized_answers[:, :-1]  # 마지막 토큰 제외 (MAX_LENGTH - 1)
    },
    {
        'outputs': tokenized_answers[:, 1:]   # 시작 토큰 제외 (MAX_LENGTH - 1)
    },
))

dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

print("✅ 데이터셋 구축 성공!")

패딩 완료! 데이터의 크기(shape): (11823, 40)
✅ 데이터셋 구축 성공!


In [32]:
# 데이터셋 구축 코드 재점검
# [루브릭 3번: 병렬 데이터 구축의 적절성 재확인]
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 텐서플로우 데이터셋으로 변환 (슬라이싱 범위 주의!)
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': tokenized_questions,
        'dec_inputs': tokenized_answers[:, :-1] # 마지막 한 칸을 자름 (시작 토큰 포함)
    },
    tokenized_answers[:, 1:] # 첫 번째(시작 토큰)를 자름
))

dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [33]:
# Step 4. 모델 구성하기
# 트랜스포머의 핵심 구조인 멀티 헤드 어텐션과 인코더, 디코더를 구현합니다.
import tensorflow as tf

# [루브릭 2번: 데이터 타입 충돌 해결 버전]

# 1. 포지셔널 인코딩 (수정됨: tf.cast 추가로 데이터 타입 일치화)
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        # [핵심 수정]: 입력을 확실히 float32 밀집 텐서로 변환하여 계산 에러 방지
        inputs = tf.cast(inputs, tf.float32)
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

# --- Step 4의 나머지 부분(MultiHeadAttention, encoder_layer 등)은 이전과 동일합니다 ---
# (코드의 가독성을 위해 생략하지만, 실제 셀에는 이전 답변의 나머지 구조가 다 있어야 합니다)

def transformer(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")
    
    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='enc_padding_mask')(inputs)
    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask, output_shape=(1, None, None), name='look_ahead_mask')(dec_inputs)
    dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='dec_padding_mask')(inputs)

    # [수정]: mask_zero=False (기본값) 확인 및 데이터 타입 명시
    enc_emb = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    enc_emb *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    enc_outputs = PositionalEncoding(vocab_size, d_model)(enc_emb)
    
    for i in range(num_layers):
        enc_outputs = encoder_layer(units=dff, d_model=d_model, num_heads=num_heads, dropout=dropout, name=f"encoder_layer_{i}")(inputs=[enc_outputs, enc_padding_mask])

    dec_emb = tf.keras.layers.Embedding(vocab_size, d_model)(dec_inputs)
    dec_emb *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    dec_outputs = PositionalEncoding(vocab_size, d_model)(dec_emb)
    
    for i in range(num_layers):
        dec_outputs = decoder_layer(units=dff, d_model=d_model, num_heads=num_heads, dropout=dropout, name=f"decoder_layer_{i}")(inputs=[dec_outputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [34]:
# 4-1
# [루브릭 2번: 안정적인 수렴을 위한 마스킹 손실 함수]
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    # 0(패딩)인 부분은 loss 계산에서 제외하도록 마스킹
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [ ]:
# [루브릭 2번: 모델 컴파일 및 50회 학습 진행]

# 1. 학습률 스케줄러 설정 (D_MODEL=256 기준)
# D_MODEL이 선언되지 않았을 경우를 대비해 256으로 고정합니다.
D_MODEL = 256
learning_rate = CustomSchedule(D_MODEL, warmup_steps=1000) 
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# 2. 모델 컴파일 (공부 방법 설정)
# loss_function이 메모리에 있어야 합니다. 
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

print("✅ 모델 준비 완료! 이제 50회 학습을 시작합니다. 숫자가 올라가는지 확인해주세요.")

# 3. 진짜 학습 시작
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

print("✅ 50회 학습이 모두 완료되었습니다! 이제 Step 5 셀을 실행해서 답변을 확인해보세요.")

✅ 모델 준비 완료! 이제 50회 학습을 시작합니다. 숫자가 올라가는지 확인해주세요.
Epoch 1/50


2026-02-05 06:12:01.613396: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 524288000 exceeds 10% of free system memory.


  1/185 ━━━━━━━━━━━━━━━━━━━━ 10:14 3s/step - accuracy: 0.0000e+00 - loss: 1.9802

2026-02-05 06:12:02.992467: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 524288000 exceeds 10% of free system memory.


  2/185 ━━━━━━━━━━━━━━━━━━━━ 3:55 1s/step - accuracy: 0.0000e+00 - loss: 1.9514 

...

  3/185 ━━━━━━━━━━━━━━━━━━━━ 3:51 1s/step - accuracy: 0.0000e+00 - loss: 1.9466

2026-02-05 06:12:05.497171: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 524288000 exceeds 10% of free system memory.


  4/185 ━━━━━━━━━━━━━━━━━━━━ 3:44 1s/step - accuracy: 0.0000e+00 - loss: 1.9386

2026-02-05 06:12:06.656809: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 524288000 exceeds 10% of free system memory.


 78/185 ━━━━━━━━━━━━━━━━━━━━ 2:08 1s/step - accuracy: 0.0097 - loss: 1.8505

In [35]:
# Step 5. 모델 평가하기
# 이제 학습된 모델이 사용자의 질문에 답변을 생성하는 예측 함수를 만듭니다.
# [루브릭 1번: 한국어 입력에 대해 한국어로 답변하는 함수 구현]

def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 1. 인코더 입력 생성 (정수 인코딩 + 패딩)
    enc_input = [s.bos_id()] + s.EncodeAsIds(sentence) + [s.eos_id()]
    enc_input = pad_sequences([enc_input], maxlen=MAX_LENGTH, padding='post')
    
    # 2. 디코더 입력 초기화 (시작 토큰 주입)
    output_sequence = tf.expand_dims([s.bos_id()], 0)

    # [중요] 모든 입력을 텐서 형태로 통일하여 ValueError 방지
    enc_input = tf.convert_to_tensor(enc_input, dtype=tf.int32)
    output_sequence = tf.cast(output_sequence, tf.int32)

    # 3. 단어 생성 루프
    for i in range(MAX_LENGTH):
        # 모델 예측 (인코더 입력과 현재까지의 디코더 입력 전달)
        predictions = model(inputs=[enc_input, output_sequence], training=False)
        
        # 마지막 타임스텝의 결과만 추출
        predictions = predictions[:, -1:, :]
        
        # 가장 높은 확률을 가진 단어 ID 선택
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 종료 토큰(</s>)이 나오면 예측 중단
        if tf.equal(predicted_id, s.eos_id()):
            break

        # 생성된 단어를 디코더 입력 시퀀스에 결합
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    # 4. 정수 시퀀스를 다시 문장으로 변환
    result = s.DecodeIds(output_sequence.numpy().squeeze().tolist())
    return result

# --- 테스트 코드 ---
print("챗봇 테스트를 시작합니다.")
questions_test = ['안녕하세요', '배고파', '고민이 있어', '오늘 날씨 어때?']

for q in questions_test:
    print(f"질문: {q} -> 답변: {decoder_inference(q)}")

챗봇 테스트를 시작합니다.
질문: 안녕하세요 -> 답변: 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심 점심
질문: 배고파 -> 답변: 되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠되겠죠
질문: 고민이 있어 -> 답변: 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골 골
질문: 오늘 날씨 어때? -> 답변: 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상 현상


✅ 데이터셋이 새롭게 구축되었습니다. 이제 학습을 다시 시도해보세요!
